<a href="https://colab.research.google.com/github/Ichigoz/Mineria/blob/main/AnaMarchena_Asignacion_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**World University Rankings 2023**
Ana Marchena [28309134]

![300 años de la UCV](https://media.meer.com/attachments/7da603ad432ef2ff10feaafcd8a94681960354b7/store/fill/1722/969/7500d742884636ee5cb111fcf3be43fb163a09164dba5464cd407a0e5022/El-proximo-22-de-diciembre-la-primera-universidad-de-Venezuela-cumplira-300-anos-de-fundada.jpg)

Para realizar el análisis de preprocesamiento del dataset designado (https://www.kaggle.com/datasets/alitaqi000/world-university-rankings-2023), en un inicio, tendremos que exportar por medio de un url nuestro archivo de datos csv ubicado en drive:

In [ ]:
import pandas as pd
import numpy as np

url = f'https://docs.google.com/spreadsheets/d/1OJn2ORGDtXKzNE1YrhxWycvaX8rB1VOuYi5G-xbprSI/export?format=csv'
df = pd.read_csv(url)
df.head()

,University Rank,Name of University,Location,No of student,No of student per staff,International Student,Female:Male Ratio,OverAll Score,Teaching Score,Research Score,Citations Score,Industry Income Score,International Outlook Score
0,1,University of Oxford,United Kingdom,"20,965",10.6,42%,48 : 52,96.4,92.3,99.7,99.0,74.9,96.2
1,2,Harvard University,United States,"21,887",9.6,25%,50 : 50,95.2,94.8,99.0,99.3,49.5,80.5
2,3,University of Cambridge,United Kingdom,"20,185",11.3,39%,47 : 53,94.8,90.9,99.5,97.0,54.2,95.8
3,3,Stanford University,United States,"16,164",7.1,24%,46 : 54,94.8,94.2,96.7,99.8,65.0,79.8
4,5,Massachusetts Institute of Technology,United States,"11,415",8.2,33%,40 : 60,94.2,90.7,93.6,99.8,90.9,89.3


Como podremos visualizar ahora con nuestros datos ya cargados o también por medio de una importacion del archivo a un modelo excel para tener un mejor manejo de los datos, podemos observar que nos encontramos con un gran conjunto de problemas funcionales en nuestro data set, entre los cuales podemos listar y transformar tal que:

1. Comenzando el analisis de los datos celda por celda de izquierda a derecha, nos encontramos primero con una inconsistencia en el apartado de raango de las universidades, ya que tiene valores muy variados yendo desde simbolos, numeros enteros y rangos. Esta falta de uniformidad puede impedir realizar cualquier análisis cuantitativo y por ello la solución aplicada consistió en un proceso estandarización donde se unificaron todos los datos faltantes e irrelevantes bajo la etiqueta Unknown, y se transformaron los rangos numéricos extrayendo su límite inferior para definir un rango unico.

In [ ]:
def fix_rank(rank):
    val = str(rank).strip()
    if val.lower() == 'reporter' or val == '-' or val == 'nan' or val == '':
        return 'Unknown'

    if '-' in val:
        return val.split('-')[0].strip()
    if '–' in val:
        return val.split('–')[0].strip()
    return val

df['University Rank'] = df['University Rank'].apply(fix_rank)
print(df)

     University Rank                     Name of University        Location  \
0                  1                   University of Oxford  United Kingdom   
1                  2                     Harvard University   United States   
2                  3                University of Cambridge  United Kingdom   
3                  3                    Stanford University   United States   
4                  5  Massachusetts Institute of Technology   United States   
...              ...                                    ...             ...   
2336         Unknown     University of the West of Scotland             NaN   
2337         Unknown                  University of Windsor             NaN   
2338         Unknown            University of Wolverhampton             NaN   
2339         Unknown                University of Wuppertal             NaN   
2340         Unknown    Xi’an Jiaotong-Liverpool University             NaN   

     No of student  No of student per staff Interna

2. Asimismo siguiendo con la evaluacion se denoto que casi al final del archivo una gran cantidad de registros presentaban una falta de datos esenciales como el nombre de la universidad a evaluar, por lo mismo se definio que era esencial que aquellas celdas que no tienen un identificador claro (Name of University) fueran filtradas.

In [ ]:
df.dropna(subset=['Name of University'], inplace=True)
print(df)

     University Rank                     Name of University        Location  \
0                  1                   University of Oxford  United Kingdom   
1                  2                     Harvard University   United States   
2                  3                University of Cambridge  United Kingdom   
3                  3                    Stanford University   United States   
4                  5  Massachusetts Institute of Technology   United States   
...              ...                                    ...             ...   
2336         Unknown     University of the West of Scotland             NaN   
2337         Unknown                  University of Windsor             NaN   
2338         Unknown            University of Wolverhampton             NaN   
2339         Unknown                University of Wuppertal             NaN   
2340         Unknown    Xi’an Jiaotong-Liverpool University             NaN   

     No of student  No of student per staff Interna

3. Siguiendo con la actividad observamos que hay muchos valores vacios o nulos dentro de la seccion location, por lo mismo para facilitar los futuros analisis comparativos por ubicación de las universidades se decidio reemplazar los datos nulos por Unknown, estandarizando las columnas de ubicación.

In [ ]:
df['Location'] = df['Location'].fillna('Unknown')
print(df)

     University Rank                     Name of University        Location  \
0                  1                   University of Oxford  United Kingdom   
1                  2                     Harvard University   United States   
2                  3                University of Cambridge  United Kingdom   
3                  3                    Stanford University   United States   
4                  5  Massachusetts Institute of Technology   United States   
...              ...                                    ...             ...   
2336         Unknown     University of the West of Scotland         Unknown   
2337         Unknown                  University of Windsor         Unknown   
2338         Unknown            University of Wolverhampton         Unknown   
2339         Unknown                University of Wuppertal         Unknown   
2340         Unknown    Xi’an Jiaotong-Liverpool University         Unknown   

     No of student  No of student per staff Interna

4. En el No. de estudiante se denoto que habia un error en el tipo de dato con el cual se estaban guardando las cantidades, por lo mismo se decidio aplicar una tranformación para definir estos mismos como un entero, además luego de ello también pudo verse que varios de los números resultantes terminaron siendo muy pequeños en comparación con otras universidades, y con un valor anormal de estudiantes, por ello se designo un rango por el cual al ser menor que una cantidad específica el número de estudiantes pasara de ser una cantidad poco fiable a un unkown.

In [ ]:
df['No of student'] = (df['No of student'].astype(str).str.replace(',', ''))
df['No of student'] = pd.to_numeric(df['No of student'], errors='coerce').fillna(0).astype(int)
print(df)

     University Rank                     Name of University        Location  \
0                  1                   University of Oxford  United Kingdom   
1                  2                     Harvard University   United States   
2                  3                University of Cambridge  United Kingdom   
3                  3                    Stanford University   United States   
4                  5  Massachusetts Institute of Technology   United States   
...              ...                                    ...             ...   
2336         Unknown     University of the West of Scotland         Unknown   
2337         Unknown                  University of Windsor         Unknown   
2338         Unknown            University of Wolverhampton         Unknown   
2339         Unknown                University of Wuppertal         Unknown   
2340         Unknown    Xi’an Jiaotong-Liverpool University         Unknown   

      No of student  No of student per staff Intern

In [ ]:
df['No of student'] = df['No of student'].astype(str).str.replace(',', '').str.strip()
def validate_students(val):
    try:
        num = float(val)
        if num < 100:
            return "Unknown"
        return str(int(num))
    except ValueError:
        return "Unknown"

df['No of student'] = df['No of student'].apply(validate_students)
print(df)

     University Rank                     Name of University        Location  \
0                  1                   University of Oxford  United Kingdom   
1                  2                     Harvard University   United States   
2                  3                University of Cambridge  United Kingdom   
3                  3                    Stanford University   United States   
4                  5  Massachusetts Institute of Technology   United States   
...              ...                                    ...             ...   
2336         Unknown     University of the West of Scotland         Unknown   
2337         Unknown                  University of Windsor         Unknown   
2338         Unknown            University of Wolverhampton         Unknown   
2339         Unknown                University of Wuppertal         Unknown   
2340         Unknown    Xi’an Jiaotong-Liverpool University         Unknown   

     No of student  No of student per staff Interna

5. Por otra parte el conjunto de porcentajes de estudiantes masculinos y femeninos se encontraba en una misma columna por lo tanto ambos datos se perdian al estar juntos y eran poco útilos de esta forma, por ello se decidio a separarlos en dos partes para así tener valores únicos por cada uno.

In [ ]:
def split_gender(ratio, pos):
    if pd.isna(ratio) or ':' not in str(ratio):
        return "Unknown"
    return str(ratio).split(':')[pos].strip()

df['Female_Percentage'] = df['Female:Male Ratio'].apply(lambda x: split_gender(x, 0))
df['Male_Percentage'] = df['Female:Male Ratio'].apply(lambda x: split_gender(x, 1))
df.drop('Female:Male Ratio', axis=1, inplace=True)
df.head()

,University Rank,Name of University,Location,No of student,No of student per staff,International Student,OverAll Score,Teaching Score,Research Score,Citations Score,Industry Income Score,International Outlook Score,Female_Percentage,Male_Percentage
0,1,University of Oxford,United Kingdom,20965,10.6,42%,96.4,92.3,99.7,99.0,74.9,96.2,48,52
1,2,Harvard University,United States,21887,9.6,25%,95.2,94.8,99.0,99.3,49.5,80.5,50,50
2,3,University of Cambridge,United Kingdom,20185,11.3,39%,94.8,90.9,99.5,97.0,54.2,95.8,47,53
3,3,Stanford University,United States,16164,7.1,24%,94.8,94.2,96.7,99.8,65.0,79.8,46,54
4,5,Massachusetts Institute of Technology,United States,11415,8.2,33%,94.2,90.7,93.6,99.8,90.9,89.3,40,60


6. Por ultimo podemos notar igualmente que existe una inconsistencia en el puntaje global de las universidades ya que pasan de un numero entero a una serie de rangos, por lo mismo se decidio transformar el rango en el promedio del mismo para obtener una variable representativa.

In [ ]:
def parse_overall(score):
    if '–' in str(score):
        low, high = str(score).split('–')
        return (float(low) + float(high)) / 2
    return pd.to_numeric(score, errors='coerce')

df['OverAll Score'] = df['OverAll Score'].apply(parse_overall)
df.tail()

,University Rank,Name of University,Location,No of student,No of student per staff,International Student,OverAll Score,Teaching Score,Research Score,Citations Score,Industry Income Score,International Outlook Score,Female_Percentage,Male_Percentage
2336,Unknown,University of the West of Scotland,Unknown,Unknown,NaN,NaN,36.6,24.1,15.5,61.5,37.9,76.8,Unknown,Unknown
2337,Unknown,University of Windsor,Unknown,Unknown,NaN,NaN,36.6,35.1,29.4,34.5,44.2,88.7,Unknown,Unknown
2338,Unknown,University of Wolverhampton,Unknown,Unknown,NaN,NaN,36.6,18.2,14.3,68.8,37.3,72.0,Unknown,Unknown
2339,Unknown,University of Wuppertal,Unknown,Unknown,NaN,NaN,36.6,26.4,26.7,52.8,52.1,47.6,Unknown,Unknown
2340,Unknown,Xi’an Jiaotong-Liverpool University,Unknown,Unknown,NaN,NaN,36.6,17.8,14.8,68.2,38.2,72.4,Unknown,Unknown
